In [16]:
def get_url(endpoint: str) -> str:
    return f"https://miftaahinstitute.neolms.com/api/v2/{endpoint}?api_key=34549ae698ce0b382e904b7a4c79a44a9da06059e7ec4f4cea7a"

In [17]:
import requests
import json
import pandas as pd
import time

# Get Users

In [18]:
rsp = requests.get(get_url("users"))
df_users = pd.DataFrame(rsp.json()['data'])

df_users['full_name'] = df_users['first_name'] + ' ' + df_users['last_name']
df_users = df_users.rename(columns={"id": "student_user_id"})
df_users = df_users[['student_user_id', 'email', 'full_name', 'joined_at']]

## Get Classes

In [19]:
rsp = requests.get(get_url("classes"))
df_classes = pd.DataFrame(rsp.json()['data'])
df_classes = df_classes.rename(columns={"id": "class_id"})
df_classes = df_classes[['class_id', 'name', 'credits', 'start_at', 'finish_at']]

## Classes Enrolled

In [20]:
classes_enrolled_df = pd.DataFrame()
for i, row in df_users.iterrows():
    url = get_url(f"users/{row.student_user_id}/classes_enrolled")
    data = requests.get(url).json().get("data")
    if data:
        cur_df = pd.DataFrame(data)
        classes_enrolled_df = pd.concat([classes_enrolled_df, cur_df], ignore_index=True)
    else:
        print(f"ERROR fetching classes_enrolled for {row}")

ERROR fetching classes_enrolled for student_user_id                    11236499
email                                  None
full_name                        Admin User
joined_at          2023-02-14T11:24:42.000Z
Name: 0, dtype: object
ERROR fetching classes_enrolled for student_user_id                    11236521
email                                  None
full_name                    Natalie Staton
joined_at          2023-02-14T11:24:46.000Z
Name: 1, dtype: object
ERROR fetching classes_enrolled for student_user_id                     11303258
email              hassan.jibril23@gmail.com
full_name                      Hassan Jibril
joined_at           2023-03-21T19:39:11.000Z
Name: 3, dtype: object


/var/folders/mc/1hb24k_n6d5_w0sbwl1w81640000gn/T/ipykernel_48617/2536451740.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  classes_enrolled_df = pd.concat([classes_enrolled_df, cur_df], ignore_index=True)


In [21]:
classes_enrolled_df = classes_enrolled_df.drop_duplicates(subset=['id'])
classes_enrolled_df = classes_enrolled_df[['id', 'student_user_id', 'class_id', 'enrolled_at', 'enroll_type', 'percent', 'grade', 'progress', 'started_at', 'completed_at', 'unenrolled']]

In [22]:
merged_df = classes_enrolled_df.merge(df_classes,on='class_id').merge(df_users,on='student_user_id')
merged_df = merged_df.drop_duplicates(subset=['id'])
merged_df.to_csv('NEO.csv')

In [23]:
classes_enrolled_df.shape

(127, 11)